In [13]:
import os.path
!pip install datasets transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


  Obtaining dependency information for kmedoids from https://files.pythonhosted.org/packages/6e/b4/457e552aa362cc746fbf0c461ab1a9bd7794db3321f82586601769f27112/kmedoids-0.5.3.1-cp312-cp312-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 kB 9.3 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Setup

In [2]:
%load_ext autoreload
%autoreload 2

device='mps'
from transformers import ClapModel, ClapFeatureExtractor, AutoTokenizer
model = ClapModel.from_pretrained("laion/clap-htsat-unfused", use_safetensors=True).to(device)
feature_extractor: ClapFeatureExtractor = ClapFeatureExtractor.from_pretrained("laion/clap-htsat-unfused")
tokenizer = AutoTokenizer.from_pretrained("laion/clap-htsat-unfused")


# Run

In [56]:
import torchaudio
import torchaudio.transforms as T

filename = '/Users/damian/2.current/clapSlice/outputs/Anderson .Paak - Come Down (Official Video).mp3'
bpm = 98.4 #97.62396675
chunk_beats = 16
waveform, sampling_rate = torchaudio.load(filename)

if sampling_rate != feature_extractor.sampling_rate:
    resampler = T.Resample(sampling_rate,  feature_extractor.sampling_rate, dtype=waveform.dtype)
    waveform, sampling_rate = resampler(waveform), feature_extractor.sampling_rate

print(sampling_rate, waveform.shape)
#inputs = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="pt")
#audio_features = model.get_audio_features(**inputs)

48000 torch.Size([2, 8381475])


In [57]:
from typing import Generator
import torch
torch.set_grad_enabled(False)
from tqdm.auto import tqdm

def wrap(idx, total):
    if idx < 0:
        return total + idx
    elif idx >= total:
        return idx - total
    else:
        return idx
    
def get_audio_chunks(waveform, chunk_size_seconds: float=0.25, 
                     window_width: float=0, include_window: bool=True
             ) -> Generator[torch.Tensor, None, None]:
    if len(waveform.shape) != 1:
        raise ValueError("waveform should have shape [num_samples]")
    chunk_size = int(8 * ((sampling_rate * chunk_size_seconds) // 8))
    window_width_samples = int(chunk_size * window_width)
    for offset in range(window_width_samples, waveform.shape[0]-window_width_samples, chunk_size):
        if include_window:
            start = wrap(offset-window_width_samples, waveform.shape[0])
            end = wrap(offset+chunk_size+window_width_samples, waveform.shape[0])
        else:
            start = offset
            end = offset + chunk_size
        yield waveform[start:end]
    

def get_audio_features(waveform: torch.Tensor, sampling_rate):
    inputs = feature_extractor(waveform, sampling_rate=sampling_rate, return_tensors="pt")
    #print(inputs.keys())
    audio_features = model.get_audio_features(input_features=inputs.input_features.to(device))
    return audio_features / torch.norm(audio_features, p=2, dim=1, keepdim=True)

def get_text_features(text: str):
    inputs = tokenizer(text, padding=True, return_tensors='pt')
    text_features =  model.get_text_features(input_ids = inputs.input_ids.to(device))
    return text_features / torch.norm(text_features, p=2, dim=1, keepdim=True)


In [58]:
import pickle
import os

chunk_size_seconds = chunk_beats*60/bpm
left_chunks = list(get_audio_chunks(waveform[0], chunk_size_seconds=chunk_size_seconds))[:-1]
right_chunks = list(get_audio_chunks(waveform[1], chunk_size_seconds=chunk_size_seconds))[:-1]
mono_chunks = [(left_chunks[i] + right_chunks[i]) / 2 for i in range(len(left_chunks))]
stereo_chunks_no_window = [torch.stack([left_chunks[index], right_chunks[index]])
    for index in range(len(left_chunks))]
features_pickle_filename = filename + f'.clap-cs{chunk_size_seconds}.pkl'
if os.path.exists(features_pickle_filename):
    with open(features_pickle_filename, 'rb') as f:
        all_features = pickle.load(f)
else:
    all_features = torch.concat([get_audio_features(chunk, sampling_rate=sampling_rate) 
                             for chunk in tqdm(mono_chunks)])
    with open(features_pickle_filename, 'wb') as f:
        pickle.dump(all_features, f)



In [59]:
from medoids_tsp import sort_tsp
sort_order = sort_tsp(all_features)

computing distance matrix
computing route


In [60]:
chunks_sorted = [torch.stack([left_chunks[index], right_chunks[index]])
    for index in sort_order]
print(chunks_sorted[0].shape)
sorted_audio = torch.cat(chunks_sorted, dim=1)
print(sorted_audio.shape)

torch.Size([2, 468288])
torch.Size([2, 7960896])


In [61]:
torchaudio.save(filename + f'-sorted-bpm{bpm}-cb{chunk_beats}.wav.mp3', sorted_audio, sample_rate=sampling_rate)

In [33]:
from typing import Literal
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            smear_width = torch.sum((logits / logits.sum()) 
                                              * torch.tensor(dynamic_smear_width_embeds[1]).to(device)).item()
            print(smear_width)
            smear_width = max(round(smear_width), 1)
            envelope = get_envelope(smear_width)
                    
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*envelope.sum()))
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smear_width = 1
spread = 4
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 6), ('intense, emotional, screaming, wailing', 1)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(s) for s, _ in dynamic_smear_width_defs]),
    [w for _, w in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}.wav', smeared, sample_rate=sampling_rate)


tensor([[    0, 40110,     2]])
tensor([0., 1., 0.])


  0%|          | 0/286 [00:00<?, ?it/s]

8.404945373535156
8.404945373535156
8.404945373535156
8.404945373535156
8.404945373535156
8.404945373535156
4.360278606414795
3.963986873626709
4.261549949645996
3.455056667327881
4.321756839752197
4.276539325714111
3.685796022415161
3.4866280555725098
2.748602867126465
3.5691187381744385
1.5581469535827637
2.2196125984191895
3.0639572143554688
2.8280022144317627
3.100942850112915
4.214722156524658
4.339332103729248
2.522089719772339
3.30554461479187
4.177463531494141
4.277876853942871
4.0346245765686035
3.8662991523742676
3.6575167179107666
3.7162070274353027
4.450263977050781
4.790956497192383
4.832127094268799
4.7884697914123535
4.475081443786621
3.5786566734313965
2.140380859375
1.190007209777832
-2.417863368988037
-2.107323169708252
6.091785430908203
11.311241149902344
-0.514557957649231
3.4570484161376953
1.1064858436584473
4.141290664672852
2.300691604614258
5.0078630447387695
3.28212833404541
3.132251739501953
5.234710693359375
4.330245494842529
-2.785930633544922
0.07736980915

In [42]:
from typing import Literal
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int], list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            logits_norm = logits / logits.sum()
            smear_width = torch.sum(
                logits_norm 
                * torch.tensor(dynamic_smear_width_embeds[1]).to(device)
            ).item() / len(dynamic_smear_width_embeds[1])
            spread = torch.sum(
                logits_norm 
                * torch.tensor(dynamic_smear_width_embeds[2]).to(device)
            ).item() / len(dynamic_smear_width_embeds[2])
            print(smear_width, spread)
            smear_width = max(round(smear_width), 1)
            spread = max(round(spread), 1)
            envelope = get_envelope(smear_width)
                    
        noclip_ramp = 200
        chunk_size_samples = original_order_chunks_no_window[0].shape[1]
        spread_envelope_in = torch.cat([torch.linspace(0, 1, noclip_ramp),
                                            torch.ones(chunk_size_samples-noclip_ramp)])
        spread_envelope_out = torch.cat([torch.ones(chunk_size_samples-noclip_ramp), 
                                                 torch.linspace(1, 0, noclip_ramp)])
        spread_envelope_flat = torch.ones(chunk_size_samples)
        spread_envelope = [spread_envelope_in] + [spread_envelope_flat] * (2*spread-1) + [spread_envelope_out]
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*envelope.sum()))
                        * spread_envelope[spread_slot+spread]
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smear_width = 1
spread = 4
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 3, 1), 
                            ('driving, rhythmic, instrumental, simple beat', 4, 4), 
                            ('intense, emotional, screaming, wailing', 2, 4)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(t) for t, _, _ in dynamic_smear_width_defs]),
    [w for _, w, _ in dynamic_smear_width_defs],
    [s for _, _, s in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape, smeared.min(), smeared.max())
smeared /= max(abs(smeared.min()), smeared.max()) 
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}-dyn3b.wav', smeared, sample_rate=sampling_rate)


tensor([[    0, 40110,     2]])
tensor([0., 1., 0.])


  0%|          | 0/143 [00:00<?, ?it/s]

1.3720874786376953 0.3579643964767456
0.7369978427886963 1.2787322998046875
1.260676383972168 0.711357593536377
1.260676383972168 0.711357593536377
1.260676383972168 0.711357593536377
1.260676383972168 0.711357593536377
1.0176369349161785 0.9716482162475586
0.9391136169433594 0.892496665318807
0.9194107055664062 0.9747852484385172
0.8587373892466227 0.875989039738973
0.91056227684021 0.9518322944641113
-0.2580081621805827 0.6263664563496908
0.10202217102050781 0.46436627705891925
-0.12888367970784506 -0.1409155527750651
0.48226292928059894 0.3545231819152832
-0.8741995493570963 -1.067533810933431
0.15381844838460287 -0.0035681724548339844
0.6014262835184733 0.49898695945739746
0.5501855214436849 0.6514197190602621
0.8346243699391683 0.7364108562469482
0.7646710872650146 0.6350129445393881
0.8430447578430176 0.5218134721120199
0.5803486506144205 0.37703696886698407
0.8631506760915121 0.809888998667399
0.8834216594696045 0.9485408465067545
1.4016183217366536 1.443535327911377
1.220994154